# Preparing input data

To use the CorrAdjust, you will need to prepare the following input data:

- Data table and additional tables with feature/sample annotations.
- One or more GMT files listing which features (e.g., genes) belong to the same reference sets (e.g., pathways).
- Configuration dict.

## Data and annotation tables

Main data table's rows and columns should represent samples and features (e.g., genes), respectively. CorrAdjust operates with Pandas data frames. Data should be normalized in a way that allows between-sample comparisons for each feature. Make sure you don't have constant features (they cannot be used for correlation analysis).

If you have more than ~100 samples, you could split the samples into training and test sets, and the module will provide methods to process them without any training/test leaks with sklearn-style interface.

Below, we use the [GTEx whole blood RNA-seq data](https://storage.googleapis.com/adult-gtex/bulk-gex/v8/rna-seq/counts-by-tissue/gene_reads_2017-06-05_v8_whole_blood.gct.gz) (this tutorial will fully reproduce Case 1 from the CorrAdjust paper). We import read counts data (pre-filtered to exclude low-expressed genes), normalize it with [median-of-ratios method](https://doi.org/10.1186/s13059-014-0550-8), and then log-transform.

In [1]:
import pandas as pd
import numpy as np
from corradjust.utils import MedianOfRatios

# Raw read counts data
df_counts = pd.read_csv(
    "input_data/GTEx_Whole_Blood/raw_counts.tsv",
    sep="\t", index_col=0
)
display(df_counts)

# Split samples into 50%/50% training and test sets
df_counts_train = df_counts.iloc[::2]
df_counts_test = df_counts.iloc[1::2]

# Normalize data using DESeq2 median of ratios algorithm
# This interface is train/test-set friendly, i.e., test data
# has no influence on training data
normalizer = MedianOfRatios()
normalizer.fit(df_counts_train)
df_norm_counts_train = normalizer.transform(df_counts_train)
df_norm_counts_test = normalizer.transform(df_counts_test)

# Log2-transform
df_data_train = np.log2(df_norm_counts_train + 1)
df_data_test = np.log2(df_norm_counts_test + 1)

ENSG00000188976.10  ENSG00000187961.13  \
GTEX-111YS-0006-SM-5NQBE                1200                 184   
GTEX-1122O-0005-SM-5O99J                 674                 117   
GTEX-1128S-0005-SM-5P9HI                1498                 742   
GTEX-113IC-0006-SM-5NQ9C                1869                 584   
GTEX-113JC-0006-SM-5O997                 652                 226   
...                                      ...                 ...   
GTEX-ZVTK-0006-SM-57WBK                 1288                 495   
GTEX-ZVZP-0006-SM-51MSW                  763                 152   
GTEX-ZVZQ-0006-SM-51MR8                 1998                 481   
GTEX-ZXES-0005-SM-57WCB                 1181                 221   
GTEX-ZXG5-0005-SM-57WCN                 1528                 766   

                          ENSG00000188290.10  ENSG00000187608.8  \
GTEX-111YS-0006-SM-5NQBE                  10                447   
GTEX-1122O-0005-SM-5O99J                  36                482   
GTEX-1128S-0005-SM-5P9HI                  60                392   
GTEX-113IC-0006-SM-5NQ9C                 456              10134   
GTEX-113JC-0006-SM-5O997                 210               7584   
...                                      ...                ...   
GTEX-ZVTK-0006-SM-57WBK                   85                579   
GTEX-ZVZP-0006-SM-51MSW                    5                319   
GTEX-ZVZQ-0006-SM-51MR8                  159               1391   
GTEX-ZXES-0005-SM-57WCB                    7                759   
GTEX-ZXG5-0005-SM-57WCN                  124                536   

                          ENSG00000131591.17  ENSG00000186891.13  \
GTEX-111YS-0006-SM-5NQBE                 303                  60   
GTEX-1122O-0005-SM-5O99J                 167                  31   
GTEX-1128S-0005-SM-5P9HI                 710                1141   
GTEX-113IC-0006-SM-5NQ9C                 891                 202   
GTEX-113JC-0006-SM-5O997                 275                 204   
...                                      ...                 ...   
GTEX-ZVTK-0006-SM-57WBK                  320                 599   
GTEX-ZVZP-0006-SM-51MSW                  260                 126   
GTEX-ZVZQ-0006-SM-51MR8                 1034                1016   
GTEX-ZXES-0005-SM-57WCB                  274                  86   
GTEX-ZXG5-0005-SM-57WCN                  583                 289   

                          ENSG00000186827.10  ENSG00000078808.16  \
GTEX-111YS-0006-SM-5NQBE                  32                5023   
GTEX-1122O-0005-SM-5O99J                  56                3917   
GTEX-1128S-0005-SM-5P9HI                 466                5158   
GTEX-113IC-0006-SM-5NQ9C                 429                4743   
GTEX-113JC-0006-SM-5O997                 184                1854   
...                                      ...                 ...   
GTEX-ZVTK-0006-SM-57WBK                  920                3557   
GTEX-ZVZP-0006-SM-51MSW                   49                4631   
GTEX-ZVZQ-0006-SM-51MR8                  785                4662   
GTEX-ZXES-0005-SM-57WCB                   65                6305   
GTEX-ZXG5-0005-SM-57WCN                  157                5351   

                          ENSG00000176022.4  ENSG00000160087.20  ...  \
GTEX-111YS-0006-SM-5NQBE                656                2291  ...   
GTEX-1122O-0005-SM-5O99J                482                2251  ...   
GTEX-1128S-0005-SM-5P9HI                101                1638  ...   
GTEX-113IC-0006-SM-5NQ9C                694                2396  ...   
GTEX-113JC-0006-SM-5O997                 84                 611  ...   
...                                     ...                 ...  ...   
GTEX-ZVTK-0006-SM-57WBK                 179                 772  ...   
GTEX-ZVZP-0006-SM-51MSW                 552                2912  ...   
GTEX-ZVZQ-0006-SM-51MR8                 257                1095  ...   
GTEX-ZXES-0005-SM-57WCB                 794       

Feature annotation table is mandatory and should have 3 columns:

1. **feature_id**: should match with columns of data and needs to be unique. For example, ENSEMBL gene IDs.
1. **feature_name**: should match feature names in the reference GMT files, allows duplicates. For example, gene symbols.
1. **feature_type**: discrete set of feature types. E.g., if you are analyzing only mRNA-seq data, put `mRNA` for all genes; if you are integrating miRNA, mRNA, or any other data type, you could use more than one type (e.g., `miRNA` and `mRNA`), see [Advanced example](advanced_run.ipynb).

Rows of feature annotation table should be **identical** to the columns of `df_data`.

In [2]:
df_feature_ann = pd.read_csv(
    "input_data/GTEx_Whole_Blood/gene_annotation.tsv",
    sep="\t", index_col=0
)
display(df_feature_ann)

feature_name feature_type
feature_id                                  
ENSG00000188976.10        NOC2L         mRNA
ENSG00000187961.13       KLHL17         mRNA
ENSG00000188290.10         HES4         mRNA
ENSG00000187608.8         ISG15         mRNA
ENSG00000131591.17     C1orf159         mRNA
...                         ...          ...
ENSG00000212907.2       MT-ND4L         mRNA
ENSG00000198886.2        MT-ND4         mRNA
ENSG00000198786.2        MT-ND5         mRNA
ENSG00000198695.2        MT-ND6         mRNA
ENSG00000198727.2        MT-CYB         mRNA

[10021 rows x 2 columns]

Finally, if you have two or more distinct sample groups (e.g., normal and tumor samples), you might provide sample annotation table, see [Advanced example](advanced_run.ipynb).

## Reference GMT files

Each reference collection of feature pairs should be represented as a separate GMT file. GMT file is tab-separated file with the following structure:

- **Column 1:** reference feature set name.
- **Column 2:** ignored, you can put any string there (e.g., `NA`).
- **Columns 3-...:** feature names (number of features can differ between rows).

You can find a [toy GMT file](https://github.com/TJU-CMC-Org/CorrAdjust/blob/master/corradjust/tests/test_data/ref_feature_sets.gmt) in the CorrAdjust GitHub repository. For the current tutorial, we downloaded Canonical Pathways and Gene Ontology databases from [MSigDB](https://www.gsea-msigdb.org/gsea/msigdb/index.jsp). 

The package will take features from each line and label all possible feature pairs as belonging to the **shared** set. Thus,
$n$ features on a line will generate $n*(n-1)/2$ shared pairs. 

One reference set might be represented by several lines. This can be useful, e.g., for miRNA-target gene pairs:

|Column 1|Column 2|Column 3|Column 4|
|-------------|--|-----|-------------|
|...|...|...|...|
|miR-X-targets|NA|miR-X|target-gene-1|
|miR-X-targets|NA|miR-X|target-gene-2|
|miR-X-targets|NA|miR-X|target-gene-3|
|...|...|...|...|

If we instead put everything in one line like this,

|Column 1|Column 2|Column 3|Column 4|Column 5|Column 6|
|-------------|--|-----|-------------|-------------|-------------|
|...|...|...|...|...|...|
|miR-X-targets|NA|miR-X|target-gene-1|target-gene-2|target-gene-3|
|...|...|...|...|...|...|

then pairs composed of two target genes will also be labeled as `miR-X-targets`,
which will be incorrect for the analysis of miRNA-mRNA targeting interactions. See [Advanced example](advanced_run.ipynb) for a miRNA-mRNA run.

## Reference feature sets configuration

Configuration dict specifies how reference sets should be handled by CorrAdjust. Below is the example of such a config for two mRNA-mRNA pathway databases. One config can contain an arbitrary number of different GMT files. See more detailed documentation in [API reference](../modules/corradjust.corradjust.rst#corradjust.corradjust.CorrAdjust).

In [3]:
ref_feature_sets = {
    "Canonical Pathways": {
        # Relative or absolute path to the GMT file.
        "path": "input_data/GMT_files/c2.cp.v2023.2.Hs.symbols.gmt",
        # Rank feature pairs by absolute correlations.
        "sign": "absolute",
        # Allowed feature pair types.
        # Should match annotation's "feature_type" column.
        "feature_pair_types": ["mRNA-mRNA"],
        # Fraction of all feature pairs to define highly ranked correlations.
        # This is paramter alpha in the CorrAdjust paper notation.
        # 0.01 is a good default value for mRNA-mRNA analysis.
        "high_corr_frac": 0.01
    },
    "Gene Ontology": {
        "path": "input_data/GMT_files/c5.go.v2023.2.Hs.symbols.gmt",
        "sign": "absolute",
        "feature_pair_types": ["mRNA-mRNA"],
        "high_corr_frac": 0.01
    }
}